# MODNet on the PBE, HSE and experimental datasets successively

In this model, we will see if training first the MODNet model on the PBE, HSE and experimental datasets and then, fine tuning the model on both the HSE and experimental datasets, and finally, fine tuning the model on the experimental dataset will improve the results.

In [1]:
from modnet.preprocessing import MODData
from modnet.models import MODNetModel
import numpy as np
import os
import copy
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [2]:
from sklearn.model_selection import KFold
from modnet.preprocessing import MODData

def shuffle_MD(data,random_state=10):
    data = copy.deepcopy(data)
    ids = data.df_targets.sample(frac=1,random_state=random_state).index
    data.df_featurized = data.df_featurized.loc[ids]
    data.df_targets = data.df_targets.loc[ids]
    data.df_structure = data.df_structure.loc[ids]
    
    return data

def MDKsplit(data,n_splits=5,random_state=10):
    data = shuffle_MD(data,random_state=random_state)
    ids = np.array(data.structure_ids)
    kf = KFold(n_splits=n_splits,shuffle=True,random_state=random_state)
    folds = []
    for train_idx, val_idx in kf.split(ids):
        data_train = MODData(data.df_structure.iloc[train_idx]['structure'].values,data.df_targets.iloc[train_idx].values,target_names=data.df_targets.columns,structure_ids=ids[train_idx])
        data_train.df_featurized = data.df_featurized.iloc[train_idx]
        #data_train.optimal_features = data.optimal_features
        
        data_val = MODData(data.df_structure.iloc[val_idx]['structure'].values,data.df_targets.iloc[val_idx].values,target_names=data.df_targets.columns,structure_ids=ids[val_idx])
        data_val.df_featurized = data.df_featurized.iloc[val_idx]
        #data_val.optimal_features = data.optimal_features

        folds.append((data_train,data_val))
        
    return folds

def MD_append(md,lmd):
    md = copy.deepcopy(md)
    for m in lmd:
        md.df_structure.append(m.df_structure)
        md.df_targets.append(m.df_targets)
        md.df_featurized.append(m.df_featurized)
    return md

In [3]:
md_exp = MODData.load('exp_gap_all')
md_exp.df_targets.columns = ['gap']
md_pbe = MODData.load('pbe_gap.zip')
md_pbe.df_targets.columns = ['gap']
md_hse = MODData.load('hse_gap.zip')
md_hse.df_targets.columns = ['gap']

md_joint = MODData.load('exp_pbe_joint')
md_joint.df_targets = md_joint.df_targets.drop(columns='pbe_gap')
md_joint.df_targets.columns = ['gap']


If you use the ChemEnv tool for your research, please consider citing the following reference(s) :
David Waroquiers, Xavier Gonze, Gian-Marco Rignanese, Cathrin Welker-Nieuwoudt, Frank Rosowski,
Michael Goebel, Stephan Schenk, Peter Degelmann, Rute Andre, Robert Glaum, and Geoffroy Hautier,
"Statistical analysis of coordination environments in oxides",
Chem. Mater., 2017, 29 (19), pp 8346-8360,
DOI: 10.1021/acs.chemmater.7b02766



INFO:root:Loaded <modnet.preprocessing.MODData object at 0x7f5ef42fe3a0> object, created with modnet version <=0.1.7
INFO:root:Loaded <modnet.preprocessing.MODData object at 0x7f5ef430f130> object, created with modnet version <=0.1.7
INFO:root:Loaded <modnet.preprocessing.MODData object at 0x7f5ef430f700> object, created with modnet version 0.1.8~develop
INFO:root:Loaded <modnet.preprocessing.MODData object at 0x7f5e6b94bee0> object, created with modnet version <=0.1.7


In [4]:
k = 5
random_state = 202010
folds = MDKsplit(md_joint,n_splits=k,random_state=random_state)
maes_ph1 = np.ones(5)
maes_ph2 = np.ones(5)
maes = np.ones(5)
for i,f in enumerate(folds):
    train = f[0]
    test = f[1]
    train.feature_selection(n=-1, use_precomputed_cross_nmi=True)
    '''fpath = 'trainA_{}_{}'.format(random_state,i+1)
    if os.path.exists(fpath):
        train = MODData.load(fpath)
    else:
        train.feature_selection(n=-1, use_precomputed_cross_nmi=True)
        train.save(fpath)
    '''     
    # assure no overlap
    assert len(set(train.df_targets.index).intersection(set(test.df_targets.index))) == 0
    
    #phase 1
    md = MD_append(train,[md_pbe,md_hse])
    
    model = MODNetModel([[['gap']]],{'gap':1})
    model.fit_preset(md,verbose=0)
    
    pred = model.predict(test)
    true = test.df_targets
    error = pred-true
    error = error.drop(pred.index[((pred['gap']).abs()>20)]) # drop unrealistic values: happens extremely rarely
    mae = np.abs(error.values).mean()
    print('mae_ph1')
    print(mae)
    maes_ph1[i] = mae
    
    # phase2
    md = MD_append(train,[md_hse])
    rlr = ReduceLROnPlateau(monitor="loss", factor=0.5, patience=20, verbose=1, mode="auto", min_delta=0)
    es = EarlyStopping(monitor="loss", min_delta=0.001, patience=300, verbose=1, mode="auto", baseline=None,restore_best_weights=True)
    model.fit(md,lr=0.01, epochs = 1000, batch_size = 64, loss='mae', callbacks=[rlr,es], verbose=0)
    
    pred = model.predict(test)
    true = test.df_targets
    error = pred-true
    error = error.drop(pred.index[((pred['gap']).abs()>20)]) # drop unrealistic values: happens extremely rarely
    mae = np.abs(error.values).mean()
    print('mae_ph2')
    print(mae)
    maes_ph2[i] = mae
    
        # phase2
    rlr = ReduceLROnPlateau(monitor="loss", factor=0.5, patience=20, verbose=1, mode="auto", min_delta=0)
    es = EarlyStopping(monitor="loss", min_delta=0.001, patience=300, verbose=1, mode="auto", baseline=None,restore_best_weights=True)
    model.fit(train,lr=0.01, epochs = 1000, batch_size = 64, loss='mae', callbacks=[rlr,es], verbose=0)
    
    pred = model.predict(test)
    true = test.df_targets
    error = pred-true
    error = error.drop(pred.index[((pred['gap']).abs()>20)]) # drop unrealistic values: happens extremely rarely
    mae = np.abs(error.values).mean()
    print('mae')
    print(mae)
    maes[i] = mae 

INFO:root:Loaded DeBreuck2020Featurizer featurizer.
INFO:root:Loaded DeBreuck2020Featurizer featurizer.
INFO:root:Loaded DeBreuck2020Featurizer featurizer.
INFO:root:Loaded DeBreuck2020Featurizer featurizer.
INFO:root:Loaded DeBreuck2020Featurizer featurizer.
INFO:root:Loaded DeBreuck2020Featurizer featurizer.
INFO:root:Loaded DeBreuck2020Featurizer featurizer.
INFO:root:Loaded DeBreuck2020Featurizer featurizer.
INFO:root:Loaded DeBreuck2020Featurizer featurizer.
INFO:root:Loaded DeBreuck2020Featurizer featurizer.
INFO:root:Loading cross NMI from 'Features_cross' file.
INFO:root:Starting target 1/1: gap ...
INFO:root:Computing mutual information between features and target...
INFO:root:Computing optimal features...
INFO:root:Selected 50/1102 features...
INFO:root:Selected 100/1102 features...
INFO:root:Selected 150/1102 features...
INFO:root:Selected 200/1102 features...
INFO:root:Selected 250/1102 features...
INFO:root:Selected 300/1102 features...
INFO:root:Selected 350/1102 features

mae_ph1
0.38398065221902195


INFO:root:Compiling model...
INFO:root:Fitting model...



Epoch 00029: ReduceLROnPlateau reducing learning rate to 0.004999999888241291.

Epoch 00062: ReduceLROnPlateau reducing learning rate to 0.0024999999441206455.

Epoch 00098: ReduceLROnPlateau reducing learning rate to 0.0012499999720603228.

Epoch 00128: ReduceLROnPlateau reducing learning rate to 0.0006249999860301614.

Epoch 00154: ReduceLROnPlateau reducing learning rate to 0.0003124999930150807.

Epoch 00186: ReduceLROnPlateau reducing learning rate to 0.00015624999650754035.

Epoch 00226: ReduceLROnPlateau reducing learning rate to 7.812499825377017e-05.

Epoch 00288: ReduceLROnPlateau reducing learning rate to 3.9062499126885086e-05.

Epoch 00323: ReduceLROnPlateau reducing learning rate to 1.9531249563442543e-05.

Epoch 00402: ReduceLROnPlateau reducing learning rate to 9.765624781721272e-06.

Epoch 00448: ReduceLROnPlateau reducing learning rate to 4.882812390860636e-06.

Epoch 00503: ReduceLROnPlateau reducing learning rate to 2.441406195430318e-06.

Epoch 00548: ReduceLROnPl

INFO:root:Compiling model...
INFO:root:Fitting model...


...

mae_ph1
0.3931253466225687


INFO:root:Compiling model...
INFO:root:Fitting model...



Epoch 00045: ReduceLROnPlateau reducing learning rate to 0.004999999888241291.

Epoch 00091: ReduceLROnPlateau reducing learning rate to 0.0024999999441206455.

Epoch 00156: ReduceLROnPlateau reducing learning rate to 0.0012499999720603228.

Epoch 00222: ReduceLROnPlateau reducing learning rate to 0.0006249999860301614.

Epoch 00284: ReduceLROnPlateau reducing learning rate to 0.0003124999930150807.

Epoch 00330: ReduceLROnPlateau reducing learning rate to 0.00015624999650754035.

Epoch 00411: ReduceLROnPlateau reducing learning rate to 7.812499825377017e-05.

Epoch 00450: ReduceLROnPlateau reducing learning rate to 3.9062499126885086e-05.

Epoch 00513: ReduceLROnPlateau reducing learning rate to 1.9531249563442543e-05.

Epoch 00549: ReduceLROnPlateau reducing learning rate to 9.765624781721272e-06.

Epoch 00595: ReduceLROnPlateau reducing learning rate to 4.882812390860636e-06.

Epoch 00655: ReduceLROnPlateau reducing learning rate to 2.441406195430318e-06.
Restoring model weights fr

INFO:root:Compiling model...
INFO:root:Fitting model...


mae_ph2
0.38706452932173363
Epoch 1/1000
22/22 [==============================] - 0s 2ms/step - loss: 0.1350 - mae: 0.1350
Epoch 2/1000
22/22 [==============================] - 0s 2ms/step - loss: 0.1106 - mae: 0.1106
Epoch 3/1000
22/22 [==============================] - 0s 2ms/step - loss: 0.1066 - mae: 0.1066
Epoch 4/1000
22/22 [==============================] - 0s 2ms/step - loss: 0.1052 - mae: 0.1052
Epoch 5/1000
22/22 [==============================] - 0s 2ms/step - loss: 0.0951 - mae: 0.0951
Epoch 6/1000
22/22 [==============================] - 0s 3ms/step - loss: 0.0817 - mae: 0.0817
Epoch 7/1000
22/22 [==============================] - 0s 2ms/step - loss: 0.0804 - mae: 0.0804
Epoch 8/1000
22/22 [==============================] - 0s 2ms/step - loss: 0.0753 - mae: 0.0753
Epoch 9/1000
22/22 [==============================] - 0s 3ms/step - loss: 0.0858 - mae: 0.0858
Epoch 10/1000
22/22 [==============================] - 0s 2ms/step - loss: 0.0925 - mae: 0.0925
Epoch 11/1000
22/22 [

In [5]:
maes_ph1.mean()

0.3582885108427475

In [6]:
maes_ph2.mean()

0.3514319399171989

In [7]:
maes

array([0.38030861, 0.35762825, 0.29331173, 0.33223507, 0.38530031])

In [8]:
maes.mean()

0.34975679551875166

#### Conclusion

~4% impovement